## ***beyond the DAG*** - patterns in more complicated workflows

### Concepts
- **Flow of Flows**: parent orchestrator patterns and subflows

- **Breaking the DAG**: changing the control flow on the fly

## Extending the `nike_flow` from earlier
Now imagine we have many URLs pointing to different Nike shoes that we like. We can create an instance of our `nike_flow` for each `url`, which wrapped in a new function we call `parent_flow`:

```python
@flow
def parent_flow(urls: List[str], budget: int) -> None:
    for url in urls:
        nike_flow(url, budget)
```

In Prefect language, this is what we call a [subflow](https://orion-docs.prefect.io/concepts/flows/#subflows) pattern. Using such a pattern, we can dynamically change the execution of the `parent_flow` based on the results we get from each `nike_flow` instance (subflow) ***during runtime***. This what we mean by *"breaking the DAG"* since representing workflows as DAGs requires knowing the execution plan ***before runtime***.



## The updated workflow

In [ ]:
from bs4 import BeautifulSoup
from prefect import flow, task
from typing import List
import requests
import re

In [ ]:
@task(retries=3, retry_delay_seconds=10)
def find_nike_price(url: str) -> int:
    k = requests.get(url).text
    soup = BeautifulSoup(k,'html.parser')
    price_string = soup.find('div', {"class":"product-price"}).text.replace(' ','')
    price = re.search('[0-9]+', price_string).group(0)
    return int(price)

In [ ]:
@task
def inBudget(price: int, budget: int) -> bool:
    return price <= budget

In [ ]:
@task
def save(url: str, to: str = 'shoes_in_my_budget.md') -> None:
    with open(to, 'w') as f:
        link = f"[{url.split('/t/')[-1]}]({url})\n\n"
        f.write(link)

In [ ]:
@flow
def nike_flow(url: str, budget: int) -> None:
    price = find_nike_price(url)
    
    if inBudget(price, budget).result():
        save(url)

In [ ]:
@flow
def parent_flow(urls: List[str], budget: int) -> None:
    for url in urls:
        nike_flow(url, budget)

In [ ]:
urls = [
    "https://www.nike.com/t/air-max-270-womens-shoes-Pgb94t/AH6789-601",
    "https://www.nike.com/t/air-max-terrascape-90-mens-shoes-R6r8hB/DH2973-100",
    "https://www.nike.com/t/pegasus-trail-3-gore-tex-mens-running-shoes-HG005k/DR0137-200"
]

budget = 150

parent_flow(urls, budget)

## What did it do?
Let's look at the output found in the [shoes_in_my_budget](shoes_in_my_budget.md) markdown file...


We see the links to the URLs pointing to shoes that have a `price` that is less than our `budget`.

### Benefits of using subflow patterns when applicable

- Ability to orchestrate many instances of a complex, stateful workflow without duplicating work

- Maintain visibility into each of the subflow processes via the UI

<hr>

### Discussion: What else might we want to use a subflow pattern for?

#### Q&A
